**Computes path usage using map matching technology (nrel/mappymatch)**

Map matching is a computation intensive task, and can give imprecise results when GPS data wasn't recorded at high enough frequency.

**Input**: "sources/data.csv" csv using the following columns:
* `transportation_mode`: used mode of transport for the trip, only walking (11) is considered here
* `start_time`: datetime of begining of trip
* `end_time`: datetime of end of trip
* `trace_gps`: list of (lon, lat) tuples
* `longitude`, `latitude`: coords of starting point (also included as first point of `trace_gps`)

**Input**: "sources/paris.geojson" Shape representation of Paris, to filter results to the inner city and improve performance

**Input/Output**: "sources/paris_walk_nxmap.json" Can also be used as an input to avoid fetching OSM data and generating the NxMap every time. It can be generated as an output from the initial NxMap generation.

**Output**: geoJSON files "../static/data/walk.geojson" containing path shapes with the following metadata:
* `road_id`: An identifier for the path, expressed as two points
* `count`: Total number of trips on the path segment
* `stroke`: A color representation of `count`, from green to red

For anonymity, there must be at least 5 trips in a given path for data to be considered


In [ ]:
import pandas as pd
from mappymatch.constructs.trace import Trace
from mappymatch.utils.plot import plot_trace
from mappymatch.constructs.geofence import Geofence
from mappymatch.maps.nx.nx_map import NxMap, NetworkType
from mappymatch.utils.plot import plot_map
from mappymatch.matchers.lcss.lcss import LCSSMatcher
from mappymatch.utils.plot import plot_matches
from mappymatch.utils.plot import plot_path
from mappymatch.utils.crs import LATLON_CRS, XY_CRS
from shapely.geometry import Point
import geopandas as gpd
import folium
import json


In [ ]:
geofence = Geofence.from_geojson("sources/paris.geojson")

In [ ]:
%%time

#nx_map = NxMap.from_geofence(geofence, network_type=NetworkType.WALK)
#nx_map.to_file("sources/paris_walk_nxmap.json")
nx_map = NxMap.from_file("sources/paris_walk_nxmap.json")

In [ ]:
#plot_map(nx_map)

In [ ]:
matcher = LCSSMatcher(nx_map)

In [ ]:
df = pd.read_csv("sources/data.csv")

In [ ]:
df_in_paris = df[df.apply(lambda x: Point(x.longitude, x.latitude).within(geofence.geometry), axis=1)]

In [ ]:
df_in_paris = df_in_paris[df_in_paris['transportation_mode'] == 11]

In [ ]:
%%time
#df['trace_gps'] = df['trace_gps'].apply(json.loads)
def gps_to_trace(gps):
    t = json.loads(gps)
    ddf = pd.DataFrame(t, columns=['lon','lat'])
    ddf = ddf[ddf.apply(lambda x: Point(x.lon, x.lat).within(geofence.geometry), axis=1)]
    trace = Trace.from_dataframe(ddf, lat_column='lat', lon_column='lon')
    return trace
df_in_paris['trace'] = df_in_paris['trace_gps'].apply(gps_to_trace)

In [ ]:
df_in_paris

In [ ]:
#ddf = pd.DataFrame(df.iloc[11012]['trace_gps'], columns=['lon','lat'])
#trace = Trace.from_dataframe(ddf, lat_column='lat', lon_column='lon')
#df.iloc[11006]
m = plot_trace(df_in_paris.iloc[0]['trace'])
c = 0
for t in df_in_paris['trace']:
    c += 1
    if c > 10:
        break
    plot_trace(t, m=m)
m

In [ ]:
%%time

#trace = df.iloc[11007]['trace']
df_in_paris_sample = df_in_paris.sample(n=4000)
match_results = matcher.match_trace_batch(df_in_paris_sample['trace'], processes=4)

In [ ]:
import folium
start_lat = 48.8915079
start_long = 2.3495425
curr_map = folium.Map(location = [start_lat, start_long], zoom_start = 17)
for i in range(0, 10):
    try:
        plot_trace(df_in_paris_sample.iloc[i]['trace'], point_color="blue", m=plot_path(match_results[i].path, crs=df_in_paris_sample.iloc[i]['trace'].crs, m=curr_map))
    except:
        pass
curr_map

In [ ]:
result_df = match_results[0].matches_to_dataframe()
result_df.head()

In [ ]:
path_df = match_results[0].path_to_dataframe()
path_df.head()

In [ ]:
df_combined = match_results[0].path_to_dataframe()
for match in match_results[1:]:
    df_combined = pd.concat([df_combined, match.path_to_dataframe()], ignore_index=True)
road_counts = df_combined['road_id'].value_counts().reset_index()
road_counts.columns = ['road_id', 'count']
df_combined = df_combined.merge(road_counts, on='road_id')
df_combined


In [ ]:
df_combined_filter = df_combined[df_combined['count']>4]
gdf = gpd.GeoDataFrame(df_combined_filter, geometry=df_combined_filter.geom, crs=XY_CRS)
gdf = gdf.to_crs(LATLON_CRS)

In [ ]:
# Export as geojson
import branca.colormap as cm

def convert_geojson(gdf, file):
    # cleanup
    gdf["road_id"] = gdf["road_id"].apply(str)
    gdf["geom"] = None

    # Colorize
    colormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=gdf["count"].max())

    gdf["stroke"] = gdf["count"].apply(lambda x: colormap(x)[:-2])

    # Export
    gdf.to_file(file, driver="GeoJSON")

convert_geojson(gdf, "../static/data/walk.geojson")

In [ ]:
m = folium.Map(location=[start_lat, start_long], zoom_start=13)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
# other mapping code (e.g. lines, markers etc.)
folium.GeoJson(gdf, style_function=lambda f: {"color": f['properties']['stroke']}).add_to(m)
folium.LayerControl().add_to(m)
m